# Corona Health - Stress Study

## Purpose of this Notebook
- Clean the dataset and save cleaned version
- Get an statistical overview
    - How many users?
    - How many assessments (= filled out questionnaires)?
    - Date range of the dataset?
    - Potential target for classification?
    - Potential features for classification?
    
    

#### Statistical overview

In [81]:
# imports
import pandas as pd
from datetime import date, datetime

In [82]:
# read in dataframe
# assuming current working directory is the folder (your/local/path/UsAs/src/d01_analysis)
df = pd.read_csv('../../data/d01_raw/ch/22-07-01_rki_stress_followup.csv')

In [83]:
# create result dict to save to disk
result = dict()

In [84]:
# Number of unique users
print('Number of user:\t', df.user_id.nunique())
result['n_users'] = df.user_id.nunique()

Number of user:	 357


In [85]:
# number of unique answers
print('Number of assessments:\t', df.answer_id.nunique())
result['n_assessments'] = df.answer_id.nunique()

Number of assessments:	 3111


In [86]:
# date range
form = '%Y-%m-%d %H:%M:%S'
date_start = df.created_at.min()
result['First assessment from'] = date_start
date_start = datetime.strptime(date_start, form)
date_end = df.created_at.max()
result['Latest assessment from'] = date_end
date_end = datetime.strptime(date_end, form)



print('Start:\t', date_start)
print('End:\t', date_end)

delta = date_end.date()-date_start.date()

print('\nDate Range in')
print('Years:\t', delta.days/365)
print('Months:\t', delta.days/12)
print('Days:\t', delta.days)


result['Time range in days'] = delta.days

Start:	 2020-12-19 07:59:15
End:	 2022-02-03 21:19:21

Date Range in
Years:	 1.126027397260274
Months:	 34.25
Days:	 411


#### Distribution of filled out questionnaires

In [87]:
bins = [0, 1, 2, 3, 5, 10, 100, 1000]
ser = pd.cut(df.user_id.value_counts(), bins = bins).value_counts().sort_index()
ser
result.update(dict(ser))

**78 users have one filled out follow up assessment, 52 have two filled out, ...92 users have 10 or more filled out follow up assessments.**

In [98]:
print('Statistical overview')
result

Statistical overview


{'n_users': 357,
 'n_assessments': 3111,
 'First assessment from': '2020-12-19 07:59:15',
 'Latest assessment from': '2022-02-03 21:19:21',
 'Time range in days': 411,
 Interval(0, 1, closed='right'): 78,
 Interval(1, 2, closed='right'): 54,
 Interval(2, 3, closed='right'): 33,
 Interval(3, 5, closed='right'): 52,
 Interval(5, 10, closed='right'): 48,
 Interval(10, 100, closed='right'): 92,
 Interval(100, 1000, closed='right'): 0}

In [88]:
# read in codebook and reduce to columns and rows of interest
cb = pd.read_excel('../../data/d00_helpers/codebook/ch/rki_stress.xlsx', sheet_name='FollowUp', header=5)
cb = cb[cb.elementtype=='question']
cb = cb.iloc[:, :30]

C:\Users\joa24jm\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
